<a href="https://colab.research.google.com/github/sunflowcoco/SS_Classification/blob/master/LSTM_CNN_VPN_800_2way.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import keras
import sys
from keras.datasets import mnist

from keras.models import Sequential

from keras.layers import Input, Dense, Dropout, Flatten, Concatenate, Reshape, LSTM
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop  
from keras import backend as K

import numpy as np  
import os
import time

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score,accuracy_score

np.random.seed(1337)
from keras.utils import np_utils  
from keras.models import Model

#DNN setting
batch_size = 256
num_classes = 6
epochs = 100

In [0]:
###############################################################################
#If equall to 'yes', then load from Bin file
FAST_LOAD = 'yes'
#postfix
FILE_TYPE = '21_40d_104685r_12711'#'756d_less'#'1134d'

#Feature column
COLUMN = 800
#K-Fold
K = 10
k = 1
# if len(sys.argv)>=2:
# 	k = int(sys.argv[1])
# if len(sys.argv)>=3:
# 	K = int(sys.argv[2])	

path = '/content/drive/SS/'
  
#Log setting
LOG_FLAG = 1
log_str = ''
logfilename = path+ '/log/'+os.path.basename(sys.argv[0])+'_'+str(K)+'_'+str(k)+'.txt'
log_str = log_str + 'Start: '+time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+'\r\n'
	
# src_file = '../data/sample_combinnativeflat21_40d_104685r_12711.txt'
# bin_file = '../data/sample_combinnativeflat21_40d_104685r_800.bin'
# src_file =path + 'data/sample_combinnativeflat21_40d_134685r_12522.txt'
# bin_file=path + 'data/sample_combinnativeflat21_40d_134685r_800.bin'
  
src_file =path + 'data/sample_combinnativeflat21_40d_5112r_12316_VPN.txt'
bin_file=path + 'data/sample_combinnativeflat21_40d_5112r_12316_VPN.bin'

In [27]:
sample = []
if FAST_LOAD=='yes':
	sample = np.fromfile(bin_file, dtype=np.double)
	sample = sample.reshape(-1, COLUMN+1)
else:
	orig = (np.loadtxt(src_file, dtype=np.str, delimiter=","))[0:104685,40:840].astype(np.double).reshape(-1, COLUMN)
	label = (np.loadtxt(src_file, dtype=np.str, delimiter=","))[0:104685,-1].astype(np.double).reshape(-1, 1)-1
	
	print(orig.shape)
	print(label.shape)
	
	sample = np.hstack((orig, label))
	np.random.shuffle(sample)
	sample.tofile(bin_file)

	
gridLen = int(len(sample)*1.0/K)
s_index = (k-1)*gridLen
e_index = k*gridLen

testX = sample[s_index:e_index, :-1]
testY = sample[s_index:e_index, -1]

if k==1:
	trainX = sample[e_index:, :-1]
	trainY = sample[e_index:, -1]
elif k==K:
	trainX = sample[0:s_index, :-1]
	trainY = sample[0:s_index, -1]
else:	
	trainX = np.vstack((sample[0:s_index, :-1], sample[e_index:, :-1]))
	trainY = np.hstack((sample[0:s_index, -1], sample[e_index:, -1]))

trainYBackup = trainY
testYBackup = testY

print ('Train Size:'+str(len(trainY)))
print ('Test Size:'+str(len(testY)))
###############################################################################

Train Size:4601
Test Size:511


In [0]:
img_rows, img_cols = 8, 40


trainX=trainX*255
testX=testX*255

trainX1 = trainX[:, :320]
trainX1 = trainX1.reshape(-1, img_rows, img_cols, 1)
trainX1_T=np.zeros((len(trainX1),img_cols,img_rows))
for i in range(len(trainX)):
	trainX1_T[i,:,:]=np.transpose(trainX1[i,:,:])

trainX2 = trainX[:, 320:]
trainX2 = trainX2.reshape(-1, 12, 40, 1)

testX1 = testX[:, :320]
testX1=testX1.reshape(-1,img_rows,img_cols,1)
testX1_T=np.zeros((len(testX1),img_cols,img_rows))
for i in range(len(testX1)):
	testX1_T[i,:,:]=np.transpose(testX1[i,:,:])


testX2 = testX[:, 320:]
testX2 = testX2.reshape(-1, 12, 40, 1)

trainX1_T = trainX1_T.reshape(-1, img_cols,img_rows, 1)  
testX1_T = testX1_T.reshape(-1, img_cols,img_rows, 1)

trainY=np_utils.to_categorical(trainY, num_classes)
testY=np_utils.to_categorical(testY, num_classes)

# input_shape = (img_rows, img_cols, 1)
input_shape = (img_cols, img_rows, 1)

del trainX
del testX
del sample

In [29]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='bestmodel_yyw_800.h5',monitor='val_loss',verbose=1,save_best_only=True)

input_data1 = Input(shape=input_shape)
reshape=Reshape(target_shape=(img_cols,img_rows))(input_data1)
lstm=LSTM(100)(reshape)
print(lstm)

#flat1 = Flatten()(lstm)

input_data2 = Input(shape=(12, 40, 1))
conv2 = Conv2D(32, kernel_size=(2, 2), activation='relu')(input_data2)
#conv2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv2 = Conv2D(64, (2, 2), activation='relu')(conv2)
conv2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flat2 = Flatten()(conv2)
print(flat2)
concatenated_tensor = Concatenate(axis=1)([lstm, flat2])

conv_output = Dense(128, activation='relu')(concatenated_tensor)
full = Dense(64, activation='relu')(conv_output)
full = Dense(32, activation='relu')(full)
full = Dense(16, activation='relu')(full)
full = Dropout(0.2)(full)
full_output = Dense(num_classes, activation='softmax')(full)

model = Model(inputs=[input_data1, input_data2], outputs=full_output)

rmsprop=RMSprop(lr=0.001,rho=0.9,epsilon=1e-08,decay=0.00) 
model.compile(loss=keras.losses.categorical_crossentropy,
			  optimizer=rmsprop,#keras.optimizers.Adadelta(),
			  metrics=['accuracy'])

history=model.fit([trainX1_T, trainX2], trainY,
		  batch_size=batch_size,
		  epochs=epochs,
		  verbose=1,
		  validation_data=([testX1_T, testX2], testY),
		  callbacks=[checkpoint])



#model.save(os.path.basename(sys.argv[0])+'_model'+'_'+FILE_TYPE+'.h5')
test_rst = model.predict([testX1_T,testX2])

test_label = np.argmax(test_rst, axis=1).astype(np.int32)
#confusion_matrix = np.zeros((num_classes,num_classes))
testY = testYBackup.astype(np.int32)

####################评估训练效果####################		
print(history.history,file=open('history_800','w'))
cf=confusion_matrix(testY,test_label)
pre=precision_score(testY,test_label,average=None)
rec=recall_score(testY,test_label,average=None)
f1_s=f1_score(testY,test_label,average=None)

print(cf)
print(pre)
print(rec)
print(f1_s)


acc=accuracy_score(testY,test_label)
print(acc)

Tensor("lstm_5/TensorArrayReadV3:0", shape=(?, 100), dtype=float32)
Tensor("flatten_5/Reshape:0", shape=(?, ?), dtype=float32)
Train on 4601 samples, validate on 511 samples
Epoch 1/100
4601/4601 [==============================] - 3s 720us/step - loss: 1.6510 - acc: 0.3573 - val_loss: 1.4465 - val_acc: 0.4442

Epoch 00001: val_loss improved from inf to 1.44652, saving model to bestmodel_yyw_800.h5
Epoch 2/100
4601/4601 [==============================] - 2s 392us/step - loss: 1.4355 - acc: 0.4186 - val_loss: 1.3950 - val_acc: 0.3933

Epoch 00002: val_loss improved from 1.44652 to 1.39502, saving model to bestmodel_yyw_800.h5
Epoch 3/100
4601/4601 [==============================] - 2s 393us/step - loss: 1.3560 - acc: 0.4521 - val_loss: 1.2762 - val_acc: 0.5323

Epoch 00003: val_loss improved from 1.39502 to 1.27618, saving model to bestmodel_yyw_800.h5
Epoch 4/100
4601/4601 [==============================] - 2s 389us/step - loss: 1.2309 - acc: 0.5188 - val_loss: 1.1590 - val_acc: 0.5910
